# Part 2 Logistic Regression

In this part, we will implement a logistic regression classifier and train it with [fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) using stochastic gradient descent (SGD).

First, download the dataset.


In [93]:
%load_ext autoreload
%autoreload 2

from typing import Tuple
import requests
import gzip
import numpy as np
import os
from dataclasses import dataclass

from src.logistic_regression import DataLoader, LogisticRegresssionClassifier
from src.logger import Logger


train_labels_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz"
train_images_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"
test_labels_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz"
test_images_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz"

os.makedirs("data", exist_ok=True)


def download_and_load(url: str, name: str, kind: str, offset: int) -> np.ndarray:
    file_path = f"data/{kind}_{name}"
    if not os.path.exists(file_path):
        response = requests.get(url)
        with open(file_path, "wb") as f:
            f.write(response.content)
    with gzip.open(file_path, "rb") as lbpath:
        return np.frombuffer(lbpath.read(), dtype=np.uint8, offset=offset)


train_labels = download_and_load(train_labels_url, "train", "labels", 8)
train_data = download_and_load(train_images_url, "train", "images",
                               16).reshape(len(train_labels), 784)
test_labels = download_and_load(test_labels_url, "test", "labels", 8)
test_data = download_and_load(test_images_url, "test", "images", 16).reshape(len(test_labels), 784)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Implementation

In this part, we will complete the ```LogisticRegresssionClassifier``` and ```DataLoader``` classes under the "src/logistic_regression.py" file. The classifier has two main methods, ```predict``` and ```fit```, similar to scikit API. Please take a look at ```fit``` method which is already completed. This method contains the main loop of the learning process.

We start the implementation with the data loader. Since we are using SGD, we need to partition the data into batches. Also, at the start of every epoch (a full pass of the data), we shuffle the data by randomly permuting it.

> Complete ```DataLoader``` by implementing ```shuffle``` and ```__next__``` methods.

**Note:** You may want to read this [article](http://www.trytoprogram.com/python-programming/python-iterators/) about ```__iter__``` method  

In [94]:
train_loader = DataLoader(train_data, train_labels, batch_size=32)
iterator = iter(train_loader)
batch_data, batch_label = next(iterator)
assert(batch_data.shape == (32, 784))
assert(batch_label.shape == (32,))

Now, we complete ```LogisticRegresssionClassifier``` starting with ```_initialize``` method. This method initiates the weights and biases of the classifier.

> Complete ```_initialize``` methods.


In [95]:
classifier = LogisticRegresssionClassifier(784, 10)
assert(classifier.weights.shape == (784, 10))
assert(classifier.bias.shape == (10,))

### Loss function

In the classification task, we use conditional class probabilities $P(Y\vert X, \theta)$ where $\theta$ denotes the parameters. We use maximum likelihood estimation (MLE) to find the best fit for the training data. However, contrary to Linear Regression models, in Logistic Regression, it is not straightforward to calculate the best fitting parameter $\theta$ with non-iterative methods due to non-linearity at the end of the function. Therefore, we use Stochastic Gradient Descent to iteratively maximize the likelihood of the data. Please read [chapters 5.5 and 5.9 of Deep Learning Book](https://www.deeplearningbook.org/contents/ml.html) for details.

Since SGD minimizes the objective and log is a monotonic function, we can use negative log-likelihood (NLL) loss to maximize the likelihood.

> Complete ```nll_loss``` methods.


In [96]:
pred_probs = np.ones((1, 5)) /5 
label = np.array([3])

assert(np.allclose(classifier.nll_loss(pred_probs, label), -np.log(pred_probs[0, 3])))



### Prediction

Next, we implement ```predict``` method to obtain conditional class probabilities $P(Y \vert X, \theta)$. In order to do that, we need to implement Softmax function that models a categorical distribution over the classes.

Before moving to implementation, calculate the jacobian of the softmax function $f(\bm{x}): \mathcal{R}^{n} \rightarrow \mathcal{R}^{n}$. You can upload an image of your solution or write it in latex.

> Calculate the jacobian of the softmax function

<!-- ![solution](Jacobian_Softmax_Function.png) -->
<img src="Jacobian_Softmax_Function.png" alt="drawing" high="200" width="1000"/>

> Complete ```softmax``` method.

In [97]:
assert(np.allclose(classifier.softmax(np.ones((1, 5))), np.ones((1, 5)) / 5))

> Complete ```predict``` method.

In [99]:
#assert(classifier.predict(next(iterator)[0]).shape == (32, 10))

In [100]:
next(iterator)[0].shape

(32, 784)

Following that, we can implement accuracy metric functions.

> Complete ```accuracy``` and ```confusion_matrix``` methods.

In [101]:
assert(classifier.accuracy(np.array([0, 1, 0, 0]), np.array([0, 1, 0, 1])) == 0.75)


In [102]:

assert((classifier.confusion_matrix(np.array([0, 1, 0, 0]), np.array([0, 1, 0, 1]),2) == np.array([[2, 1], [0, 1]])).all())

Not: confusion_matrix fonksiyonuna model.n_classes inputunu ekledim. labels ve predictions inputlarının confusion_matrix'in boyutunu belirlerken yeterli olamayacağı durumlar olduğunu düşündüm. Örneğin labels veya predictions larıma bakararak unique değerlerin sayısına göre confusion_matrix boyutunu belirleyebilirdim fakat labels veya predictions'da olmayan durumlar olabilirdi. Bu sebeple generic bir yapıya dönüştürmek istedim.

### Learning

Now that we completed the prediction and data loader parts, we can start completing the requisite methods for ```fit```. In order to update parameters, we need to calculate gradients of the loss function with respect to weights and biases.

> Implement ```nll_gradients``` method

In [103]:
label = np.array(1, dtype=np.int32)

inccorect_pred = np.zeros((1, 10), dtype=np.float32)
inccorect_pred[0, 0] = 1.0

correct_pred = np.zeros((1, 10), dtype=np.float32)
correct_pred[0, 1] = 1.0



inputs = np.ones((1, 784), dtype=np.float32)

weight_grad, bias_grad = classifier.nll_gradients(probs=correct_pred, inputs=inputs, labels=label)
assert((weight_grad == 0).all())
assert((bias_grad == 0).all())

weight_grad, bias_grad = classifier.nll_gradients(probs=inccorect_pred, inputs=inputs, labels=label)

assert((weight_grad[:, 0] == 1.0).all() and (weight_grad[:, 1] == -1.0).all() and (weight_grad[:, 2:] == 0.0).all())
assert((bias_grad[0] == 1.0) and (bias_grad[1] == -1.0) and (bias_grad[2:] == 0.0).all())

Finally, we need to implement the ```update``` method that updates the parameters with the given gradient and l2 regularization.

> Implement ```update``` method

In [104]:
prev_weights = classifier.weights.copy()
prev_bias = classifier.bias.copy()
classifier.update((np.ones_like(prev_weights), np.ones_like(prev_bias)), 1.0, 0.0)

assert(np.allclose(classifier.weights, prev_weights -1.0))
assert(np.allclose(classifier.bias, prev_bias -1.0))


Now that we finalized our implementation, we can train our classifier on the fashion MNIST dataset. Split the train data into train and eval sets and create ```train_loader``` and ```eval_loader```.

> Implement ```split_dataset``` function

In [105]:
def split_dataset(data: np.ndarray, label: np.ndarray, batch_size: int, train_ratio: float = 0.9
                  ) -> Tuple[DataLoader, DataLoader]:
    """ Split the data into train and eval sets

    Args:
        data (np.ndarray): Data array of shape (B, D)
        label (np.ndarray): Label array os shape (B)
        batch_size (int): Batch size of the dataloaders
        train_ratio (float): Ratio of the train sample size to overall sample size 

    Returns:
        Tuple[DataLoader, DataLoader]: Train and Eval Dataloaders
    """
    z = int(data.shape[0] * train_ratio)
    
    train_data, eval_data = data[:z], data[z:]
    train_labels, eval_labels = label[:z], label[z:]
    
    train_loader = DataLoader(train_data, train_labels, batch_size=32)
    eval_loader = DataLoader(eval_data, eval_labels, batch_size=32)
    
    return train_loader, eval_loader

@dataclass
class Hyperparameters():
    train_eval_ratio: float = 0.8
    batch_size: int = 32
    learning_rate: float = 1e-3
    l2_coeff: float = 1e-1
    epoch: int = 20


hyperparams = Hyperparameters()

train_loader, eval_loader = split_dataset(train_data, train_labels, hyperparams.batch_size)
logger = Logger(smooth_window_len=100, verbose=False, live_figure_update=True)
logger.render()


    'data': [{'mode': 'markers+lines',
              'name': 'accuracy',
       …

When you run the cell above, if you do not encounter any exception, you must see two empty plots side by side. When model ```fit``` function calls loggers ```log_iteration``` and ```log_epoch``` methods, these plots will be updated with the new values (given that logger is initailized with ```live_figure_update=True```). You can also use ```verbose``` option that allows logger to prints out the logs.

#### Start Training

You can start model training by running the cell below.

In [114]:
model = LogisticRegresssionClassifier(784, 10)
model.fit(train_loader,
          eval_loader,
          hyperparams.learning_rate,
          hyperparams.l2_coeff,
          hyperparams.epoch,
          logger)


Hopefully, classifier fits the data with more than 80% evaluation accuracy. If so, you can run the model with the test data using the Test Loader and observe the confusion matrix in the test data.

> Implement ```test_classifier``` function

In [115]:
def test_classifier(data_loader: DataLoader, model: LogisticRegresssionClassifier) -> np.ndarray:
    """ Run the model with test data loader and return confusion matrix

    Args:
        data_loader (DataLoader): Data loader of the test data
        model (LogisticRegresssionClassifier): Trained classifier

    Returns:
        np.ndarray: Confusion matrix
    """
    
    cm = np.zeros((model.n_classes,model.n_classes)) 
    for iter_index, (data, labels) in enumerate(data_loader):
        probs = model.predict(data)
        predictions = probs.argmax(axis=-1)
        cm += model.confusion_matrix(labels,predictions,model.n_classes)
    return cm


test_loader = DataLoader(test_data, test_labels, hyperparams.batch_size)
confusion_matrix = test_classifier(test_loader, model)

> Lets run the cell below to plot ```confusion_matrix```

In [116]:
logger.render_confusion_matrix(confusion_matrix)